In [1]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT
from train import get_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = ModelConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/models/scot/proportional_scot.yaml")
ETL_CONFIG = ETLConfig(name = "no_preprocessing", log1p = False, drop_unmatched_perts = True)
EMBEDDING_CONFIG = EmbeddingConfig(pert_embedding='GenePT')

SPLIT_CONFIG = DatasplitConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml")
EVAL_CONFIG = EvalConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/eval_config.yaml")  # Set this if you want to run evaluations

# Load configurations
config = Config(model_config=MODEL_CONFIG,
                 etl_config=ETL_CONFIG, 
                 datasplit_config=SPLIT_CONFIG, 
                 eval_config=EVAL_CONFIG)


#Alternatively you can initialize the config objects manually as follows:
# etl_config = ETLConfig(name = XXX, log1p = False, drop_unmatched_perts = False, ...)
# model_config = ...
# embedding_config = ...
# datasplit_config = ...
# eval_config = ...
# config = Config(etl_config, model_config, datasplit_config, eval_config)


config.etl_config.pert_embedding = 'bioBERT'
config.etl_config.drop_unmatched_perts = True
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.obsm['embedding'].shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")


2025-01-31 17:42:21,379 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/repogle_k562_essential_raw.yaml
2025-01-31 17:42:21,382 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_raw.yaml
2025-01-31 17:42:21,384 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/adamson_INCOMPLETE.yaml
2025-01-31 17:42:21,385 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_HVG.yaml
2025-01-31 17:42:21,387 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/kang.yaml
2025-01-31 17:42:21,388 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/essential_gene_knockouts_raw.yaml
2025-01-31 17:42:21,390 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNG_raw_INCOMPLET

Using device: cuda


2025-01-31 17:42:24,799 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-01-31 17:42:24,800 - INFO - Preprocessing training data
2025-01-31 17:42:24,803 - INFO - Using identity features for perturbations
2025-01-31 17:42:24,925 - INFO - Removing observations with perturbations not in the dataset as a column
2025-01-31 17:42:25,115 - INFO - Removed 189 perturbations that were not in the dataset columns and 0 perturbations that did not have an embedding for a total of 189 perturbations removed out of an initial 2058 perturbations
/orcd/data/omarabu/001/njwfish/omnicell/notebooks/../omnicell/data/loader.py:178: ImplicitModificationWarning: Setting element `.obsm['embedding']` of view, initializing view as actual.
  adata.obsm["embedding"] = adata.X.toarray().astype('float32')
2025-01-31 17:43:08,380 - INFO - Doing OOD split


Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [2]:
import numpy as np
from omnicell.models.datamodules import get_dataloader
dset, ns, dl = get_dataloader(adata, pert_ids=np.array(adata.obs[PERT_KEY].values), pert_map=pert_rep_map, collate='ot')


['k562', 'k562', 'k562', 'k562', 'k562', ..., 'k562', 'k562', 'k562', 'k562', 'k562']
Length: 10691
Categories (1, object): ['k562'] ['k562'] 1
Creating source indices
Creating target indices
Creating pert indices
Creating source and target dicts
Strata probs [1.85915765e-05 1.85915765e-05 1.85915765e-05 ... 3.62163911e-03
 3.68856878e-03 7.42175735e-03]


In [ ]:
from omnicell.models.distribute_shift import sample_pert, get_proportional_weighted_dist

synthetic_counterfactuals = {}
for stratum in dset.strata:
    synthetic_counterfactuals[stratum] = {}
    for pert in dset.unique_pert_ids:
        print(stratum, pert, (dset.source[stratum].shape, dset.target[stratum][pert].shape))
        X_ctrl = dset.source[stratum]
        X_pert = dset.target[stratum][pert]
        mean_ctrl = X_ctrl.mean(axis=0)
        mean_pert = X_pert.mean(axis=0)
        
        mean_shift = mean_pert - mean_ctrl
        logger.debug(f"Mean shift shape: {mean_shift.shape}")

        weighted_dist = get_proportional_weighted_dist(X_ctrl)
        
        preds = sample_pert(X_ctrl, weighted_dist, mean_shift, max_rejections=100)
        synthetic_counterfactuals[stratum][pert] = preds


k562 AAAS ((10691, 8563), (212, 8563))
k562 AAMP ((10691, 8563), (99, 8563))
k562 AARS ((10691, 8563), (40, 8563))
k562 AARS2 ((10691, 8563), (181, 8563))
k562 AASDHPPT ((10691, 8563), (49, 8563))
k562 AATF ((10691, 8563), (298, 8563))
k562 ABCB10 ((10691, 8563), (51, 8563))
k562 ABCB7 ((10691, 8563), (199, 8563))
k562 ABCE1 ((10691, 8563), (249, 8563))
k562 ABCF1 ((10691, 8563), (329, 8563))
k562 ABHD11 ((10691, 8563), (49, 8563))
k562 ABHD17A ((10691, 8563), (95, 8563))
k562 ABT1 ((10691, 8563), (107, 8563))
k562 ACD ((10691, 8563), (77, 8563))
k562 ACIN1 ((10691, 8563), (191, 8563))
k562 ACTB ((10691, 8563), (168, 8563))
k562 ACTL6A ((10691, 8563), (55, 8563))
k562 ACTR10 ((10691, 8563), (116, 8563))
k562 ACTR1A ((10691, 8563), (143, 8563))
k562 ACTR1B ((10691, 8563), (206, 8563))
k562 ACTR2 ((10691, 8563), (236, 8563))
k562 ACTR3 ((10691, 8563), (118, 8563))
k562 ACTR6 ((10691, 8563), (108, 8563))
k562 ACTR8 ((10691, 8563), (252, 8563))
k562 ADAT2 ((10691, 8563), (85, 8563))
k562 A